In [23]:
friends_activity = main.get_friends_activity_json()

In [47]:
import psycopg2

connection = psycopg2.connect(dbname="FriendsActivity",
                             user="postgres",
                             password="whatthe",
                             host="localhost",
                             port="5432")

def get_table_details(connection):
    with connection.cursor() as cursor:
        cursor.execute("""SELECT table_name FROM information_schema.tables 
                          WHERE table_schema = 'public'""")
        tables = cursor.fetchall()
        table_details = {}
        for table in tables:
            table_name = table[0]
            cursor.execute(f"SELECT column_name FROM information_schema.columns WHERE table_name = '{table_name}'")
            columns = cursor.fetchall()
            table_details[table_name] = [col[0] for col in columns]
    return table_details

def get_sample_rows(connection, table_details):
    sample_data = {}
    with connection.cursor() as cursor:
        for table, columns in table_details.items():
            cursor.execute(f"SELECT * FROM {table} LIMIT 2")
            rows = cursor.fetchall()
            sample_data[table] = rows
    return sample_data

# get_table_details(connection)
# get_sample_rows(connection, table_details)

In [43]:

# table_details = get_table_details(connection)
# get_sample_rows(connection, table_details)

In [46]:
def find_duplicate_album_uris(connection):
    with connection.cursor() as cursor:
        cursor.execute("""SELECT album_uri, COUNT(*) 
                          FROM albums 
                          GROUP BY album_uri 
                          HAVING COUNT(*) > 1""")
        duplicates = cursor.fetchall()
    return duplicates

def remove_duplicate_albums(connection):
    with connection.cursor() as cursor:
        cursor.execute("""
        DELETE FROM albums
        WHERE ctid NOT IN (
            SELECT DISTINCT ON (album_uri) ctid
            FROM albums
            ORDER BY album_uri, ctid
        );
        """)
        removed_duplicates = cursor.rowcount
        connection.commit()
        print(f"Removed {removed_duplicates} duplicate rows.")


connection.rollback()
# find_duplicate_album_uris(connection)
# remove_duplicate_albums(connection)
def remove_duplicate_albums_and_update_references(connection):
    with connection.cursor() as cursor:
        # Step 1: Identify the Duplicates
        cursor.execute("""
        SELECT DISTINCT ON (album_uri) album_id, album_uri
        FROM albums
        ORDER BY album_uri, album_id;
        """)
        unique_albums = cursor.fetchall()

        total_removed = 0
        for album_id, album_uri in unique_albums:
            # Step 2: Update the References
            cursor.execute("""
            UPDATE tracks 
            SET album_id = %s 
            WHERE album_id IN (SELECT album_id FROM albums WHERE album_uri = %s AND album_id != %s);
            """, (album_id, album_uri, album_id))
            
            # Step 3: Delete the Duplicate
            cursor.execute("""
            DELETE FROM albums 
            WHERE album_uri = %s AND album_id != %s;
            """, (album_uri, album_id))
            
            total_removed += cursor.rowcount
        
        connection.commit()
        print(f"Removed {total_removed} duplicate rows.")
remove_duplicate_albums_and_update_references(connection)


Removed 2 duplicate rows.


In [3]:
def add_unique_constraint_to_album_uri(connection):
    with connection.cursor() as cursor:
        cursor.execute("ALTER TABLE albums ADD UNIQUE(album_uri);")
        connection.commit()
    print("Unique constraint added to album_uri column in albums table.")

# Let's execute the function using your database connection:

connection = psycopg2.connect(database="FriendsActivity", user="postgres", password="whatthe", host="localhost", port="5432")
add_unique_constraint_to_album_uri(connection)
connection.close()


Unique constraint added to album_uri column in albums table.


In [ ]:
connection.rollback()
find_duplicate_album_uris(connection)


In [48]:
def insert_data(connection, activity):
    with connection.cursor() as cursor:
        for entry in data:
            user = entry['user']
            track = entry['track']
            album = track['album']
            artist = track['artist']
            context = track['context']
            
            # Insert artist data
            cursor.execute("INSERT INTO artists(artist_uri, artist_name) VALUES (%s, %s) ON CONFLICT (artist_uri) DO NOTHING RETURNING artist_id", 
                           (artist['uri'], artist['name']))
            artist_id = cursor.fetchone()[0] if cursor.rowcount else None
            
            # Insert user data
            cursor.execute("INSERT INTO users(user_url, user_name, user_image_url) VALUES (%s, %s, %s) ON CONFLICT (user_url) DO NOTHING RETURNING user_id", 
                           (user['uri'], user['name'], user.get('imageUrl', '')))
            user_id = cursor.fetchone()[0] if cursor.rowcount else None
            
            # Insert album data
            cursor.execute("INSERT INTO albums(album_uri, album_name) VALUES (%s, %s) ON CONFLICT (album_uri) DO NOTHING RETURNING album_id", 
                           (album['uri'], album['name']))
            album_id = cursor.fetchone()[0] if cursor.rowcount else None
            
            # Insert track data
            cursor.execute("""INSERT INTO tracks(track_uri, track_name, track_image_url, album_id, artist_id) 
                              VALUES (%s, %s, %s, %s, %s) ON CONFLICT (track_uri) DO NOTHING RETURNING track_id""", 
                           (track['uri'], track['name'], track.get('imageUrl', ''), album_id, artist_id))
            track_id = cursor.fetchone()[0] if cursor.rowcount else None
            
            # Insert context data
            cursor.execute("""INSERT INTO context(context_uri, context_name, context_index) 
                              VALUES (%s, %s, %s) ON CONFLICT (context_uri) DO NOTHING RETURNING context_id""", 
                           (context['uri'], context['name'], context['index']))
            context_id = cursor.fetchone()[0] if cursor.rowcount else None
            
            # Insert streaming data
            if user_id and track_id and context_id:
                cursor.execute("INSERT INTO streamings(user_id, track_id, context_id, timestamp) VALUES (%s, %s, %s, %s)", 
                               (user_id, track_id, context_id, entry['timestamp']))
            
        connection.commit()

insert_data(connection, friends_activity)

InvalidColumnReference: there is no unique or exclusion constraint matching the ON CONFLICT specification


In [49]:
def add_and_print_new_data_to_postgresql(friends_data, 
                                                  postgres_db_name="FriendsActivity",
                                                  user="postgres",
                                                  password="whatthe",
                                                  host="localhost",
                                                  port="5432"):
    # Connect to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()

    for data in friends_data['friends']:
        # User data extraction and insertion
        user_uri = data['user']['uri']
        user_name = data['user']['name']
        
        # Check if 'imageUrl' key exists, if not, use an empty string
        user_image = data['user'].get('imageUrl', '')
        
        postgres_cur.execute("SELECT user_url FROM users WHERE user_url = %s", (user_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO users (user_url, user_name, user_image_url) VALUES (%s, %s, %s) ON CONFLICT (user_url) DO NOTHING",
                (user_uri, user_name, user_image)
            )
            print(f"Added user: {user_name}, URI: {user_uri}")

        # Album data extraction and insertion
        album_uri = data['track']['album']['uri']
        album_name = data['track']['album']['name']

        postgres_cur.execute("SELECT album_uri FROM albums WHERE album_uri = %s", (album_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO albums (album_uri, album_name) VALUES (%s, %s) ON CONFLICT (album_uri) DO NOTHING",
                (album_uri, album_name)
            )
            print(f"Added album: {album_name}, URI: {album_uri}")

        # Artist data extraction and insertion
        artist_uri = data['track']['artist']['uri']
        artist_name = data['track']['artist']['name']

        postgres_cur.execute("SELECT artist_uri FROM artists WHERE artist_uri = %s", (artist_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO artists (artist_uri, artist_name) VALUES (%s, %s) ON CONFLICT (artist_uri) DO NOTHING",
                (artist_uri, artist_name)
            )
            print(f"Added artist: {artist_name}, URI: {artist_uri}")

        # Track data extraction and insertion
        track_uri = data['track']['uri']
        track_name = data['track']['name']
        track_image = data['track']['imageUrl']

        postgres_cur.execute("SELECT track_uri FROM tracks WHERE track_uri = %s", (track_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                """INSERT INTO tracks (track_uri, track_name, track_image_url, album_id, artist_id) 
                   VALUES (%s, %s, %s, 
                           (SELECT album_id FROM albums WHERE album_uri = %s), 
                           (SELECT artist_id FROM artists WHERE artist_uri = %s))
                   ON CONFLICT (track_uri) DO NOTHING""",
                (track_uri, track_name, track_image, album_uri, artist_uri)
            )
           
    # Close the PostgreSQL connection
    postgres_conn.commit()
    postgres_conn.close()



add_and_print_new_data_to_postgresql(friends_activity)

InvalidColumnReference: there is no unique or exclusion constraint matching the ON CONFLICT specification


In [ ]:
def add_unique_constraint_to_album_uri(connection):
    with connection.cursor() as cursor:
        cursor.execute("ALTER TABLE albums ADD UNIQUE(album_uri);")
        connection.commit()
    print("Unique constraint added to album_uri column in albums table.")

# Now, let's execute the function using your database connection:

# connection = psycopg2.connect(database="FriendsActivity", user="postgres", password="whatthe", host="localhost", port="5432")
# add_unique_constraint_to_album_uri(connection)
# connection.close()
def check_for_duplicates_in_album_uri(connection):
    with connection.cursor() as cursor:
        cursor.execute("""
        SELECT album_uri, COUNT(*)
        FROM albums
        GROUP BY album_uri
        HAVING COUNT(*) > 1
        """)
        duplicates = cursor.fetchall()
    return duplicates

connection = psycopg2.connect(database="FriendsActivity", user="postgres", password="whatthe", host="localhost", port="5432")
duplicates = check_for_duplicates_in_album_uri(connection)
connection.close()

duplicates


: 

: 

In [26]:

import main
import psycopg2
import sqlite3

def store_user_data_to_postgresql(sqlite_db_name='friends_activity.db',
                                  postgres_db_name="FriendsActivity",
                                  user="postgres",
                                  password="whatthe",
                                  host="localhost",
                                  port="5432"):
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()

    # Creating the tables in PostgreSQL
    tables = [
        '''
        CREATE TABLE IF NOT EXISTS users (
            user_id SERIAL PRIMARY KEY,
            user_url TEXT NOT NULL,
            user_name TEXT NOT NULL,
            user_image_url TEXT
        )
        ''',
        '''
        CREATE TABLE IF NOT EXISTS albums (
            album_id SERIAL PRIMARY KEY,
            album_uri TEXT NOT NULL,
            album_name TEXT NOT NULL
        )
        ''',
        '''
        CREATE TABLE IF NOT EXISTS artists (
            artist_id SERIAL PRIMARY KEY,
            artist_uri TEXT NOT NULL,
            artist_name TEXT NOT NULL
        )
        ''',
        '''
        CREATE TABLE IF NOT EXISTS tracks (
            track_id SERIAL PRIMARY KEY,
            track_uri TEXT NOT NULL,
            track_name TEXT NOT NULL,
            track_image_url TEXT NOT NULL,
            album_id INTEGER NOT NULL REFERENCES albums(album_id),
            artist_id INTEGER NOT NULL REFERENCES artists(artist_id)
        )
        ''',
        '''
        CREATE TABLE IF NOT EXISTS context (
            context_id SERIAL PRIMARY KEY,
            context_uri TEXT NOT NULL,
            context_name TEXT NOT NULL,
            context_index INTEGER NOT NULL
        )
        ''',
        '''
        CREATE TABLE IF NOT EXISTS streamings (
            user_id INTEGER NOT NULL REFERENCES users(user_id),
            track_id INTEGER NOT NULL REFERENCES tracks(track_id),
            timestamp TEXT NOT NULL,
            context_id INTEGER NOT NULL REFERENCES context(context_id),
            UNIQUE(user_id, track_id, timestamp, context_id)  -- Add this unique constraint
        )
        '''
       
    ]

    for table in tables:
        postgres_cur.execute(table)
    postgres_conn.commit()

    # Connect to SQLite
    sqlite_conn = sqlite3.connect(sqlite_db_name)
    sqlite_cur = sqlite_conn.cursor()

    # Table primary key mapping for upsert
    primary_keys = {
        "users": "user_id",
        "albums": "album_id",
        "artists": "artist_id",
        "tracks": "track_id",
        "context": "context_id",
        "streamings": "user_id, track_id, timestamp, context_id"  # Corrected composite primary key
    }
    # # Extracting and inserting data with upsert logic
    for table_name, primary_key in primary_keys.items():
        sqlite_cur.execute(f"SELECT * FROM {table_name}")
        rows = sqlite_cur.fetchall()
        columns = [desc[0] for desc in sqlite_cur.description]
        columns_str = ", ".join(columns)
        placeholders = ", ".join(["%s"] * len(rows[0])) if rows else ""

        for row in rows:
            # If inserting into the 'streamings' table, check for NULL context_id
            if table_name == "streamings" and row[3] is None:
                # Handle NULL context_id (e.g., skip insertion or replace with a default value)
                continue  # This will skip inserting this particular row

            postgres_cur.execute(
                f"INSERT INTO {table_name} ({columns_str}) VALUES ({placeholders}) ON CONFLICT ({primary_key}) DO NOTHING",
                row
            )
        postgres_conn.commit()


# Uncomment the following line to run the function. Ensure the PostgreSQL DB details are correct.
# store_user_data_to_postgresql(friends_activity_json_example)

def fetch_data_from_postgresql(table_name,
                               postgres_db_name="FriendsActivity",
                               user="postgres",
                               password="whatthe",
                               host="localhost",
                               port="5432"):
    # Connect to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()

    # Fetch data from the specified table
    postgres_cur.execute(f"SELECT * FROM {table_name}")
    records = postgres_cur.fetchall()
    columns = [desc[0] for desc in postgres_cur.description]

    postgres_conn.close()
    return columns, records

# version 1
def add_friends_data_to_postgresql(friends_data, 
                                  postgres_db_name="FriendsActivity",
                                  user="postgres",
                                  password="whatthe",
                                  host="localhost",
                                  port="5432"):
    # Connect to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()

    for data in friends_data:
        # User data extraction and insertion
        user_uri = data['user']['uri']
        user_name = data['user']['name']
        user_image = data['user']['imageUrl']

        postgres_cur.execute(
            "INSERT INTO users (user_url, user_name, user_image_url) VALUES (%s, %s, %s) ON CONFLICT (user_url) DO NOTHING",
            (user_uri, user_name, user_image)
        )

        # Album data extraction and insertion
        album_uri = data['track']['album']['uri']
        album_name = data['track']['album']['name']

        postgres_cur.execute(
            "INSERT INTO albums (album_uri, album_name) VALUES (%s, %s) ON CONFLICT (album_uri) DO NOTHING",
            (album_uri, album_name)
        )

        # Artist data extraction and insertion
        artist_uri = data['track']['artist']['uri']
        artist_name = data['track']['artist']['name']

        postgres_cur.execute(
            "INSERT INTO artists (artist_uri, artist_name) VALUES (%s, %s) ON CONFLICT (artist_uri) DO NOTHING",
            (artist_uri, artist_name)
        )

        # Track data extraction and insertion
        track_uri = data['track']['uri']
        track_name = data['track']['name']
        track_image = data['track']['imageUrl']

        postgres_cur.execute(
            """INSERT INTO tracks (track_uri, track_name, track_image_url, album_id, artist_id) 
               VALUES (%s, %s, %s, 
                       (SELECT album_id FROM albums WHERE album_uri = %s), 
                       (SELECT artist_id FROM artists WHERE artist_uri = %s)) 
               ON CONFLICT (track_uri) DO NOTHING""",
            (track_uri, track_name, track_image, album_uri, artist_uri)
        )

        # Context data extraction and insertion
        context_uri = data['track']['context']['uri']
        context_name = data['track']['context']['name']
        context_index = data['track']['context']['index']

        postgres_cur.execute(
            "INSERT INTO context (context_uri, context_name, context_index) VALUES (%s, %s, %s) ON CONFLICT (context_uri) DO NOTHING",
            (context_uri, context_name, context_index)
        )

        # Streamings data extraction and insertion
        timestamp = data['timestamp']

        postgres_cur.execute(
            """INSERT INTO streamings (user_id, track_id, timestamp, context_id) 
               VALUES (
                   (SELECT user_id FROM users WHERE user_url = %s), 
                   (SELECT track_id FROM tracks WHERE track_uri = %s), 
                   %s, 
                   (SELECT context_id FROM context WHERE context_uri = %s)
               ) 
               ON CONFLICT (user_id, track_id, timestamp, context_id) DO NOTHING""",
            (user_uri, track_uri, timestamp, context_uri)
        )

    postgres_conn.commit()
    postgres_conn.close()


def add_and_print_new_data_to_postgresql(friends_data, 
                                         postgres_db_name="FriendsActivity",
                                         user="postgres",
                                         password="whatthe",
                                         host="localhost",
                                         port="5432"):
    # Connect to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()

    for data in friends_data['friends']:
        # User data extraction and insertion
        
        user_uri = data['user']['uri']
        user_name = data['user']['name']
        user_image = data['user']['imageUrl']
        
        postgres_cur.execute("SELECT user_url FROM users WHERE user_url = %s", (user_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO users (user_url, user_name, user_image_url) VALUES (%s, %s, %s)",
                (user_uri, user_name, user_image)
            )
            print(f"Added user: {user_name}, URI: {user_uri}")

        # Album data extraction and insertion
        album_uri = data['track']['album']['uri']
        album_name = data['track']['album']['name']

        postgres_cur.execute("SELECT album_uri FROM albums WHERE album_uri = %s", (album_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO albums (album_uri, album_name) VALUES (%s, %s)",
                (album_uri, album_name)
            )
            print(f"Added album: {album_name}, URI: {album_uri}")

        # Artist data extraction and insertion
        artist_uri = data['track']['artist']['uri']
        artist_name = data['track']['artist']['name']

        postgres_cur.execute("SELECT artist_uri FROM artists WHERE artist_uri = %s", (artist_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO artists (artist_uri, artist_name) VALUES (%s, %s)",
                (artist_uri, artist_name)
            )
            print(f"Added artist: {artist_name}, URI: {artist_uri}")

        # Track data extraction and insertion
        print(data['user']['uri'])
        track_uri = data['track']['uri']
        track_name = data['track']['name']
        track_image = data['track']['imageUrl']

        postgres_cur.execute("SELECT track_uri FROM tracks WHERE track_uri = %s", (track_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                """INSERT INTO tracks (track_uri, track_name, track_image_url, album_id, artist_id) 
                   VALUES (%s, %s, %s, 
                           (SELECT album_id FROM albums WHERE album_uri = %s), 
                           (SELECT artist_id FROM artists WHERE artist_uri = %s))""",
                (track_uri, track_name, track_image, album_uri, artist_uri)
            )
           
def add_and_print_new_data_to_postgresql(friends_data, 
                                         postgres_db_name="FriendsActivity",
                                         user="postgres",
                                         password="whatthe",
                                         host="localhost",
                                         port="5432"):
    # Connect to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()

    for data in friends_data['friends']:
        # User data extraction and insertion
        user_uri = data['user']['uri']
        user_name = data['user']['name']
        
        # Check if 'imageUrl' key exists, if not, use an empty string
        user_image = data['user'].get('imageUrl', '')
        
        postgres_cur.execute("SELECT user_url FROM users WHERE user_url = %s", (user_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO users (user_url, user_name, user_image_url) VALUES (%s, %s, %s)",
                (user_uri, user_name, user_image)
            )
            print(f"Added user: {user_name}, URI: {user_uri}")

        # Album data extraction and insertion
        album_uri = data['track']['album']['uri']
        album_name = data['track']['album']['name']

        postgres_cur.execute("SELECT album_uri FROM albums WHERE album_uri = %s", (album_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO albums (album_uri, album_name) VALUES (%s, %s)",
                (album_uri, album_name)
            )
            print(f"Added album: {album_name}, URI: {album_uri}")

        # Artist data extraction and insertion
        artist_uri = data['track']['artist']['uri']
        artist_name = data['track']['artist']['name']

        postgres_cur.execute("SELECT artist_uri FROM artists WHERE artist_uri = %s", (artist_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO artists (artist_uri, artist_name) VALUES (%s, %s)",
                (artist_uri, artist_name)
            )
            print(f"Added artist: {artist_name}, URI: {artist_uri}")

        # Track data extraction and insertion
        track_uri = data['track']['uri']
        track_name = data['track']['name']
        track_image = data['track']['imageUrl']

        postgres_cur.execute("SELECT track_uri FROM tracks WHERE track_uri = %s", (track_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                """INSERT INTO tracks (track_uri, track_name, track_image_url, album_id, artist_id) 
                   VALUES (%s, %s, %s, 
                           (SELECT album_id FROM albums WHERE album_uri = %s), 
                           (SELECT artist_id FROM artists WHERE artist_uri = %s))""",
                (track_uri, track_name, track_image, album_uri, artist_uri)
            )
           
    # Close the PostgreSQL connection
    postgres_conn.commit()
    postgres_conn.close()

# Note: As we are in an environment without psycopg2, this function can't be executed here. 
# You can use it in your local environment where psycopg2 is available.

# Uncomment to run the function with the example data
# add_friends_data_to_postgresql(friends_data)


# Example usage
# table_data = fetch_data_from_postgresql("artists")
# print(table_data)


# store_user_data_to_postgresql()


In [22]:
import psycopg2
import csv

def fetch_data_and_save_to_csv(table_name, filename,
                               postgres_db_name="FriendsActivity",
                               user="postgres",
                               password="whatthe",
                               host="localhost",
                               port="5432"):
    # Connect to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()

    # Fetch data from the specified table
    postgres_cur.execute(f"SELECT * FROM {table_name}")
    records = postgres_cur.fetchall()
    columns = [desc[0] for desc in postgres_cur.description]

    # Write data to CSV
    with open(filename, 'w', newline='') as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(columns)  # Writing the headers
        csvwriter.writerows(records)  # Writing the data

    # Close the PostgreSQL connection
    postgres_conn.close()

# Example usage
fetch_data_and_save_to_csv("users", "artists_data.csv")


UnicodeEncodeError: 'charmap' codec can't encode character '\u26e7' in position 49: character maps to <undefined>

In [71]:
import re

def convert_sqlite_to_postgresql(sqlite_sql):
    # Convert INTEGER PRIMARY KEY to SERIAL
    postgres_sql = re.sub(r"INTEGER PRIMARY KEY", "SERIAL PRIMARY KEY", sqlite_sql)
    
    # Replace back-tick (`) with double quotes (") - SQLite uses back-ticks to escape column names, PostgreSQL uses double quotes
    postgres_sql = re.sub(r"`", '"', postgres_sql)
    
    # SQLite PRAGMA statements are not used in PostgreSQL, so we'll remove them
    postgres_sql = re.sub(r"PRAGMA .+?;", "", postgres_sql)
    
    # Add a SET statement at the beginning to adjust the time zone (optional, but can be helpful)
    postgres_sql = "SET timezone='UTC';\n" + postgres_sql
    
    # Other transformations can be added as needed
    
    return postgres_sql

def main():
    file_path = "C:\\Users\\saket\\OneDrive\\Documents\\new projects\\spotify\\Project Spotify\\Project_spotify\\dump.sql"
    
    with open(file_path, "r", encoding="utf-16") as file:
        content = file.read()
        
    converted_content = convert_sqlite_to_postgresql(content)
    
    output_path = file_path.replace("dump.sql", "converted_dump.sql")
    
    with open(output_path, "w", encoding="utf-8") as file:
        file.write(converted_content)
    
    print(f"Converted SQL saved to: {output_path}")

if __name__ == "__main__":
    main()


Converted SQL saved to: C:\Users\saket\OneDrive\Documents\new projects\spotify\Project Spotify\Project_spotify\converted_dump.sql


In [6]:
import main
activity = main.get_friends_activity_json()

In [7]:
data = activity['friends']

In [14]:
# First, let's install the chardet library if you don't have it already
try:
    import chardet
except ImportError:
    !pip install chardet

import chardet

def find_file_encoding(file_path):
    # Read a small chunk of the file to determine the encoding
    with open(file_path, 'rb') as f:
        raw_data = f.read(1000)  # Read the first 1000 bytes

    # Detect the encoding
    result = chardet.detect(raw_data)
    encoding = result['encoding']

    return encoding

# Replace 'your_file_path' with the path to your file
file_path = r'C:\Users\saket\OneDrive\Documents\new projects\spotify\Project Spotify\Project_spotify\converted_dump.sql'
encoding = find_file_encoding(file_path)
print(f"The file is encoded in: {encoding}")

The file is encoded in: ascii


In [15]:
def convert_ascii_to_utf8(input_file_path, output_file_path):
    # Read the content of the input file using ASCII encoding
    with open(input_file_path, 'r', encoding='ascii') as input_file:
        ascii_content = input_file.read()

    # Write the content to the output file using UTF-8 encoding
    with open(output_file_path, 'w', encoding='utf-8') as output_file:
        output_file.write(ascii_content)

# Replace 'input_file.txt' with the path to your input file in ASCII encoding
# Replace 'output_file.txt' with the path where you want to save the UTF-8 encoded file
input_file_path = r'C:\Users\saket\OneDrive\Documents\new projects\spotify\Project Spotify\Project_spotify\converted_dump.sql'
output_file_path = 'output_file.txt'

convert_ascii_to_utf8(input_file_path, output_file_path)
print("File converted from ASCII to UTF-8.")




UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 3475: ordinal not in range(128)

In [10]:
print(file_path)

C:\Users\saket\OneDrive\Documents\new projects\spotify\Project Spotify\Project_spotify\converted_dump.sql


In [3]:
import psycopg2

# Data

# PostgreSQL connection parameters
params = {
    "dbname": "FriendsActivity",
    "user": "postgres",
    "password": "whatthe",
    "host": "localhost",
    "port": "5432"
}

# Connect to PostgreSQL
conn = psycopg2.connect(**params)
cur = conn.cursor()

for entry in data:
    added_data = []

    # Insert user
    user_uri = entry['user']['uri']
    user_name = entry['user']['name']
    user_image_url = entry['user']['imageUrl']
    cur.execute(
        "INSERT INTO users (user_url, user_name, user_image_url) VALUES (%s, %s, %s) ON CONFLICT (user_url) DO NOTHING RETURNING user_url;",
        (user_uri, user_name, user_image_url)
    )
    if cur.fetchone():
        added_data.append({'user': entry['user']})

    # Insert artist
    artist_uri = entry['track']['artist']['uri']
    artist_name = entry['track']['artist']['name']
    cur.execute(
        "INSERT INTO artists (artist_uri, artist_name) VALUES (%s, %s) ON CONFLICT (artist_uri) DO NOTHING RETURNING artist_uri;",
        (artist_uri, artist_name)
    )
    if cur.fetchone():
        added_data.append({'artist': entry['track']['artist']})

    # Insert album
    album_uri = entry['track']['album']['uri']
    album_name = entry['track']['album']['name']
    cur.execute(
        "INSERT INTO albums (album_uri, album_name) VALUES (%s, %s) ON CONFLICT (album_uri) DO NOTHING RETURNING album_uri;",
        (album_uri, album_name)
    )
    if cur.fetchone():
        added_data.append({'album': entry['track']['album']})

    # Insert track
    track_uri = entry['track']['uri']
    track_name = entry['track']['name']
    track_image_url = entry['track']['imageUrl']
    cur.execute(
        "INSERT INTO tracks (track_uri, track_name, track_image_url, album_id, artist_id) VALUES (%s, %s, %s, (SELECT album_id FROM albums WHERE album_uri=%s), (SELECT artist_id FROM artists WHERE artist_uri=%s)) ON CONFLICT (track_uri) DO NOTHING RETURNING track_uri;",
        (track_uri, track_name, track_image_url, album_uri, artist_uri)
    )
    if cur.fetchone():
        added_data.append({'track': entry['track']})

    # Insert context
    context_uri = entry['track']['context']['uri']
    context_name = entry['track']['context']['name']
    context_index = entry['track']['context']['index']
    cur.execute(
        "INSERT INTO context (context_uri, context_name, context_index) VALUES (%s, %s, %s) ON CONFLICT (context_uri) DO NOTHING RETURNING context_uri;",
        (context_uri, context_name, context_index)
    )
    if cur.fetchone():
        added_data.append({'context': entry['track']['context']})

    # Insert streaming
    timestamp = entry['timestamp']
    cur.execute(
        "INSERT INTO streamings (user_id, track_id, timestamp, context_id) VALUES ((SELECT user_id FROM users WHERE user_url=%s), (SELECT track_id FROM tracks WHERE track_uri=%s), %s, (SELECT context_id FROM context WHERE context_uri=%s)) RETURNING timestamp;",
        (user_uri, track_uri, timestamp, context_uri)
    )
    if cur.fetchone():
        added_data.append({'streaming': {'timestamp': timestamp}})

    if added_data:
        print("Added Data:", added_data)

# Commit changes
conn.commit()

# Close connection
cur.close()
conn.close()


In [62]:
def fetch_data_from_sqlite(table_name, sqlite_db_name='friends_activity.db'):
    # Connect to SQLite
    sqlite_conn = sqlite3.connect(sqlite_db_name)
    sqlite_cur = sqlite_conn.cursor()

    # Fetch data from the specified table
    sqlite_cur.execute(f"SELECT * FROM {table_name}")
    records = sqlite_cur.fetchall()

    sqlite_conn.close()
    return set(records)  # Convert to set for easier comparison

def fetch_data_from_postgres(table_name, 
                             postgres_db_name="FriendsActivity",
                             user="postgres",
                             password="whatthe",
                             host="localhost",
                             port="5432"):
    # Connect to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()

    # Fetch data from the specified table
    postgres_cur.execute(f"SELECT * FROM {table_name}")
    records = postgres_cur.fetchall()

    postgres_conn.close()
    return set(records)  # Convert to set for easier comparison

def compare_databases(table_name):
    sqlite_data = fetch_data_from_sqlite(table_name)
    postgres_data = fetch_data_from_postgres(table_name)

    # Find differences
    missing_in_postgres = sqlite_data - postgres_data
    extra_in_postgres = postgres_data - sqlite_data

    return missing_in_postgres, extra_in_postgres

# Let's test with the 'users' table as an example
missing_users, extra_users = compare_databases("albums")
len(missing_users), len(extra_users)  # Number of discrepancies for 'users' table
print(missing_users,"where the fuck the extra come from", extra_users)

{(10932, 'spotify:album:5cy3t8bW0lNzw9hjCPkRUT', 'Real Life'), (10961, 'spotify:album:5DzXM9YcY99gfgHBHOjioI', 'If Children'), (10960, 'spotify:album:1hfO7IMUNnifiQxrak3DvV', 'For the Recently Found Innocent'), (10966, 'spotify:album:6yoaegjPtXrC8fI8fQpPPl', "Magic Isn't Real"), (10967, 'spotify:album:6MeO7Ccuf5WCI12WatTCEG', 'Caught In The Act'), (10963, 'spotify:album:54Ykk8Gg8TuHjLbYvErsKh', 'Showcase'), (10959, 'spotify:album:0q16Fq67WOuXSxF3ngrbW8', 'Los Pobres'), (10968, 'spotify:album:1uzfGk9vxMXfaZ2avqwxod', '2012 - 2017'), (10958, 'spotify:album:7L6I21oEZcN6n1jRyYyV6c', 'Open Your Heart'), (10962, 'spotify:album:1pBLLI9jbR05lCW8mKBiAP', 'Gluee'), (10957, 'spotify:album:0ObRnroCwS1JQZUMOgQj3X', 'Okovi'), (10964, 'spotify:album:0tAMTTxU0e84QFhrNhQNIT', 'Mr. Fantasy'), (10965, 'spotify:album:42VUtnjwSkIRlXFiA8p7So', 'Skydrvg 1.0'), (10933, 'spotify:album:0Mk6BHv9bvolXiiit564oy', 'This Last Night...In Sodom')} where the fuck the extra come from {(10932, 'spotify:album:20knLZluvaF9

In [ ]:
params = {
    "dbname": "friendsactivity",
    "user": "myuser",
    "password": "whatthe",
    "host": "127.0.0.1",  # connecting from Windows to WSL
    "port": "5432"
}


In [66]:
j = {(10932, 'spotify:album:5cy3t8bW0lNzw9hjCPkRUT', 'Real Life'), (10961, 'spotify:album:5DzXM9YcY99gfgHBHOjioI', 'If Children'), (10960, 'spotify:album:1hfO7IMUNnifiQxrak3DvV', 'For the Recently Found Innocent'), (10966, 'spotify:album:6yoaegjPtXrC8fI8fQpPPl', "Magic Isn't Real"), (10967, 'spotify:album:6MeO7Ccuf5WCI12WatTCEG', 'Caught In The Act'), (10963, 'spotify:album:54Ykk8Gg8TuHjLbYvErsKh', 'Showcase'), (10959, 'spotify:album:0q16Fq67WOuXSxF3ngrbW8', 'Los Pobres'), (10968, 'spotify:album:1uzfGk9vxMXfaZ2avqwxod', '2012 - 2017'), (10958, 'spotify:album:7L6I21oEZcN6n1jRyYyV6c', 'Open Your Heart'), (10962, 'spotify:album:1pBLLI9jbR05lCW8mKBiAP', 'Gluee'), (10957, 'spotify:album:0ObRnroCwS1JQZUMOgQj3X', 'Okovi'), (10964, 'spotify:album:0tAMTTxU0e84QFhrNhQNIT', 'Mr. Fantasy'), (10965, 'spotify:album:42VUtnjwSkIRlXFiA8p7So', 'Skydrvg 1.0'), (10933, 'spotify:album:0Mk6BHv9bvolXiiit564oy', 'This Last Night...In Sodom')}, "where the fuck the extra come from", {(10932, 'spotify:album:20knLZluvaF9ogIjJeBEsX', "I Won't Fall To Your Love"), (10933, 'spotify:album:7IKUTIc9UWuVngyGPtqNHS', 'L.A. Woman')}
j

({(10932, 'spotify:album:5cy3t8bW0lNzw9hjCPkRUT', 'Real Life'),
  (10933,
   'spotify:album:0Mk6BHv9bvolXiiit564oy',
   'This Last Night...In Sodom'),
  (10957, 'spotify:album:0ObRnroCwS1JQZUMOgQj3X', 'Okovi'),
  (10958, 'spotify:album:7L6I21oEZcN6n1jRyYyV6c', 'Open Your Heart'),
  (10959, 'spotify:album:0q16Fq67WOuXSxF3ngrbW8', 'Los Pobres'),
  (10960,
   'spotify:album:1hfO7IMUNnifiQxrak3DvV',
   'For the Recently Found Innocent'),
  (10961, 'spotify:album:5DzXM9YcY99gfgHBHOjioI', 'If Children'),
  (10962, 'spotify:album:1pBLLI9jbR05lCW8mKBiAP', 'Gluee'),
  (10963, 'spotify:album:54Ykk8Gg8TuHjLbYvErsKh', 'Showcase'),
  (10964, 'spotify:album:0tAMTTxU0e84QFhrNhQNIT', 'Mr. Fantasy'),
  (10965, 'spotify:album:42VUtnjwSkIRlXFiA8p7So', 'Skydrvg 1.0'),
  (10966, 'spotify:album:6yoaegjPtXrC8fI8fQpPPl', "Magic Isn't Real"),
  (10967, 'spotify:album:6MeO7Ccuf5WCI12WatTCEG', 'Caught In The Act'),
  (10968, 'spotify:album:1uzfGk9vxMXfaZ2avqwxod', '2012 - 2017')},
 'where the fuck the extra come

In [ ]:
old sqlite3 database
Table users has 37 rows.
Table albums has 10966 rows.
Table artists has 6458 rows.
Table tracks has 19585 rows.
Table context has 5729 rows.
Table streamings has 36801 rows.

new PostgreSQL database
Table users has 37 rows.
Table albums has 10956 rows.
Table artists has 6450 rows.
Table tracks has 19571 rows.
Table context has 5729 rows.
Table streamings has 31251 rows.

In [56]:
import sqlite3

def get_rows_count_from_sqlite(sqlite_db_name='friends_activity.db'):
    # Connect to SQLite
    sqlite_conn = sqlite3.connect(sqlite_db_name)
    sqlite_cur = sqlite_conn.cursor()
    
    tables = ["users", "albums", "artists", "tracks", "context", "streamings"]
    count_data = {}
    
    for table in tables:
        sqlite_cur.execute(f"SELECT COUNT(*) FROM {table}")
        count = sqlite_cur.fetchone()[0]
        count_data[table] = count
    
    sqlite_conn.close()
    return count_data

# Get the rows count and print
rows_count = get_rows_count_from_sqlite()
for table, count in rows_count.items():
    print(f"Table {table} has {count} rows.")


Table users has 37 rows.
Table albums has 10966 rows.
Table artists has 6458 rows.
Table tracks has 19585 rows.
Table context has 5729 rows.
Table streamings has 36801 rows.


SyntaxError: invalid syntax (762227826.py, line 1)

In [57]:
import psycopg2

def get_rows_count_from_postgresql(postgres_db_name="FriendsActivity",
                                   user="postgres",
                                   password="whatthe",
                                   host="localhost",
                                   port="5432"):
    # Connect to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()
    
    tables = ["users", "albums", "artists", "tracks", "context", "streamings"]
    count_data = {}
    
    for table in tables:
        postgres_cur.execute(f"SELECT COUNT(*) FROM {table}")
        count = postgres_cur.fetchone()[0]
        count_data[table] = count
    
    postgres_conn.close()
    return count_data

# Get the rows count and print
rows_count = get_rows_count_from_postgresql()
for table, count in rows_count.items():
    print(f"Table {table} has {count} rows.")


Table users has 37 rows.
Table albums has 10956 rows.
Table artists has 6450 rows.
Table tracks has 19571 rows.
Table context has 5729 rows.
Table streamings has 31251 rows.


In [ ]:
the old sqlite3
Table users has 37 rows.
Table albums has 10956 rows.
Table artists has 6450 rows.
Table tracks has 19571 rows.
Table context has 5729 rows.
Table streamings has 31251 rows.
=

In [49]:
store_user_data_to_postgresql()

In [40]:
def add_friends_data_to_postgresql(friends_data, 
                                  postgres_db_name="FriendsActivity",
                                  user="postgres",
                                  password="whatthe",
                                  host="localhost",
                                  port="5432"):
    # Connect to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()

    for data in friends_data['friends']:
        # User data extraction and insertion
        # if not isinstance(data, dict):
        #     print("Unexpected data type:", type(data), data)
        #     continue

        user_uri = data['user']['uri']
        user_name = data['user']['name']
        user_image = data['user'].get('imageUrl', None)
        postgres_cur.execute("SELECT * FROM users WHERE user_url = %s", (user_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO users (user_url, user_name, user_image_url) VALUES (%s, %s, %s)",
                (user_uri, user_name, user_image)
            )
            print(f"Added User: {user_name}")

        # Album data extraction and insertion
        album_uri = data['track']['album']['uri']
        album_name = data['track']['album']['name']
        postgres_cur.execute("SELECT * FROM albums WHERE album_uri = %s", (album_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO albums (album_uri, album_name) VALUES (%s, %s)",
                (album_uri, album_name)
            )
            print(f"Added Album: {album_name}")

        # Artist data extraction and insertion
        artist_uri = data['track']['artist']['uri']
        artist_name = data['track']['artist']['name']
        postgres_cur.execute("SELECT * FROM artists WHERE artist_uri = %s", (artist_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO artists (artist_uri, artist_name) VALUES (%s, %s)",
                (artist_uri, artist_name)
            )
            print(f"Added Artist: {artist_name}")

        # Track data extraction and insertion
        track_uri = data['track']['uri']
        track_name = data['track']['name']
        track_image = data['track']['imageUrl']
        postgres_cur.execute("SELECT * FROM tracks WHERE track_uri = %s", (track_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                """INSERT INTO tracks (track_uri, track_name, track_image_url, album_id, artist_id) 
                   VALUES (%s, %s, %s, 
                           (SELECT album_id FROM albums WHERE album_uri = %s), 
                           (SELECT artist_id FROM artists WHERE artist_uri = %s))""",
                (track_uri, track_name, track_image, album_uri, artist_uri)
            )
            print(f"Added Track: {track_name}")

        # Context data extraction and insertion
        context_uri = data['track']['context']['uri']
        context_name = data['track']['context']['name']
        context_index = data['track']['context']['index']
        postgres_cur.execute("SELECT * FROM context WHERE context_uri = %s", (context_uri,))
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                "INSERT INTO context (context_uri, context_name, context_index) VALUES (%s, %s, %s)",
                (context_uri, context_name, context_index)
            )
            print(f"Added Context: {context_name}")

        # Streamings data extraction and insertion
        timestamp = data['timestamp']
        postgres_cur.execute(
            """SELECT * FROM streamings 
               WHERE user_id = (SELECT user_id FROM users WHERE user_url = %s) 
               AND track_id = (SELECT track_id FROM tracks WHERE track_uri = %s)
               AND timestamp = %s::TEXT
               AND context_id = (SELECT context_id FROM context WHERE context_uri = %s)""",
            (user_uri, track_uri, timestamp, context_uri)
        )
        if not postgres_cur.fetchone():
            postgres_cur.execute(
                """INSERT INTO streamings (user_id, track_id, timestamp, context_id) 
                   VALUES (
                       (SELECT user_id FROM users WHERE user_url = %s), 
                       (SELECT track_id FROM tracks WHERE track_uri = %s), 
                       %s, 
                       (SELECT context_id FROM context WHERE context_uri = %s)
                   )""",
                (user_uri, track_uri, timestamp, context_uri)
            )
            print(f"Added Streaming: {user_name} listened to {track_name} at {timestamp}")

    postgres_conn.commit()
    postgres_conn.close()


In [38]:
print(type(activity['friends']))
print(activity['friends'])


<class 'list'>
[{'timestamp': 1690609572062, 'user': {'uri': 'spotify:user:homestar99', 'name': 'Eliott', 'imageUrl': 'https://i.scdn.co/image/ab67757000003b82c53693eae40a7069725a0785'}, 'track': {'uri': 'spotify:track:3deWxhcgA9D5ww7YcPQltQ', 'name': 'Dinner for Two', 'imageUrl': 'http://i.scdn.co/image/ab67616d0000b273258152311d36cc59f779c3a7', 'album': {'uri': 'spotify:album:4oRaFi044yhwH77gUXbH5I', 'name': 'Love This Giant'}, 'artist': {'uri': 'spotify:artist:20vuBdFblWUo2FCOvUzusB', 'name': 'David Byrne'}, 'context': {'uri': 'spotify:playlist:4RSutFssm5BrHALNngF9hf', 'name': "That's Why God Made the Radio", 'index': 0}}}, {'timestamp': 1690726442406, 'user': {'uri': 'spotify:user:sonemic.com', 'name': 'sonemic.com', 'imageUrl': 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f'}, 'track': {'uri': 'spotify:track:7z3OJLElVcKcNty1TIjCUD', 'name': 'Vulture City', 'imageUrl': 'http://i.scdn.co/image/ab67616d0000b273046b6e3667aaf93d653a4adf', 'album': {'uri': 'spotify:al

In [5]:
activity = main.get_friends_activity_json()

In [75]:
activity['friends'][0:3]

[{'timestamp': 1690609572062,
  'user': {'uri': 'spotify:user:homestar99',
   'name': 'Eliott',
   'imageUrl': 'https://i.scdn.co/image/ab67757000003b82c53693eae40a7069725a0785'},
  'track': {'uri': 'spotify:track:3deWxhcgA9D5ww7YcPQltQ',
   'name': 'Dinner for Two',
   'imageUrl': 'http://i.scdn.co/image/ab67616d0000b273258152311d36cc59f779c3a7',
   'album': {'uri': 'spotify:album:4oRaFi044yhwH77gUXbH5I',
    'name': 'Love This Giant'},
   'artist': {'uri': 'spotify:artist:20vuBdFblWUo2FCOvUzusB',
    'name': 'David Byrne'},
   'context': {'uri': 'spotify:playlist:4RSutFssm5BrHALNngF9hf',
    'name': "That's Why God Made the Radio",
    'index': 0}}},
 {'timestamp': 1690726442406,
  'user': {'uri': 'spotify:user:sonemic.com',
   'name': 'sonemic.com',
   'imageUrl': 'https://i.scdn.co/image/ab67757000003b82642ce4dbd1cab92bc267868f'},
  'track': {'uri': 'spotify:track:7z3OJLElVcKcNty1TIjCUD',
   'name': 'Vulture City',
   'imageUrl': 'http://i.scdn.co/image/ab67616d0000b273046b6e3667aa

In [33]:
for i in activity['friends']:
    print(i)
    break

{'timestamp': 1690609572062, 'user': {'uri': 'spotify:user:homestar99', 'name': 'Eliott', 'imageUrl': 'https://i.scdn.co/image/ab67757000003b82c53693eae40a7069725a0785'}, 'track': {'uri': 'spotify:track:3deWxhcgA9D5ww7YcPQltQ', 'name': 'Dinner for Two', 'imageUrl': 'http://i.scdn.co/image/ab67616d0000b273258152311d36cc59f779c3a7', 'album': {'uri': 'spotify:album:4oRaFi044yhwH77gUXbH5I', 'name': 'Love This Giant'}, 'artist': {'uri': 'spotify:artist:20vuBdFblWUo2FCOvUzusB', 'name': 'David Byrne'}, 'context': {'uri': 'spotify:playlist:4RSutFssm5BrHALNngF9hf', 'name': "That's Why God Made the Radio", 'index': 0}}}


In [46]:
add_friends_data_to_postgresql(activity)

Added Album: I Won't Fall To Your Love
Added Artist: Nick Moore
Added Track: I Won't Fall To Your Love
Added Context: I Won't Fall To Your Love
Added Streaming: Irles Music listened to I Won't Fall To Your Love at 1690965453875
Added Context: the past
Added Streaming: maja listened to April Come She Will at 1690965777433
Added Album: L.A. Woman
Added Track: Riders on the Storm
Added Streaming: Ken Langelier listened to Riders on the Storm at 1690967011019


In [45]:
def reset_sequence_for_table(table_name, column_name, 
                             postgres_db_name="FriendsActivity",
                             user="postgres",
                             password="whatthe",
                             host="localhost",
                             port="5432"):
    # Connect to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()
    
    # Get the sequence name associated with the column (assuming standard naming convention)
    sequence_name = f"{table_name}_{column_name}_seq"
    
    # Reset the sequence
    postgres_cur.execute(f"SELECT setval('{sequence_name}', (SELECT MAX({column_name}) FROM {table_name}));")
    
    postgres_conn.commit()
    postgres_conn.close()

# Reset sequence for album_id in albums table
# reset_sequence_for_table("albums", "album_id")
def reset_all_sequences():
    tables_and_columns = [
        ("albums", "album_id"),
        ("artists", "artist_id"),
        ("tracks", "track_id"),
        ("users", "user_id"),
        ("context", "context_id")
    ]

    for table, column in tables_and_columns:
        reset_sequence_for_table(table, column)

# Now, run the function to reset sequences for all specified tables
reset_all_sequences()



In [2]:
import main
import psycopg2
import sqlite3

def store_user_data_to_postgresql(friends_activity_json, 
                                  sqlite_db_name='friends_activity.db',
                                  postgres_db_name="your_postgres_db",
                                  user="postgres",
                                  password="spotify",
                                  host="localhost",
                                  port="5432"):
    # First, let's establish a connection to PostgreSQL
    postgres_conn = psycopg2.connect(database=postgres_db_name, user=user, password=password, host=host, port=port)
    postgres_cur = postgres_conn.cursor()

    # Creating the tables in PostgreSQL
    tables = [
        '''
        CREATE TABLE IF NOT EXISTS users (
            user_id SERIAL PRIMARY KEY,
            user_url TEXT NOT NULL,
            user_name TEXT NOT NULL,
            user_image_url TEXT
        )
        ''',
        '''
        CREATE TABLE IF NOT EXISTS albums (
            album_id SERIAL PRIMARY KEY,
            album_uri TEXT NOT NULL,
            album_name TEXT NOT NULL
        )
        ''',
        '''
        CREATE TABLE IF NOT EXISTS artists (
            artist_id SERIAL PRIMARY KEY,
            artist_uri TEXT NOT NULL,
            artist_name TEXT NOT NULL
        )
        ''',
        '''
        CREATE TABLE IF NOT EXISTS tracks (
            track_id SERIAL PRIMARY KEY,
            track_uri TEXT NOT NULL,
            track_name TEXT NOT NULL,
            track_image_url TEXT NOT NULL,
            album_id INTEGER NOT NULL REFERENCES albums(album_id),
            artist_id INTEGER NOT NULL REFERENCES artists(artist_id)
        )
        ''',
        '''
        CREATE TABLE IF NOT EXISTS context (
            context_id SERIAL PRIMARY KEY,
            context_uri TEXT NOT NULL,
            context_name TEXT NOT NULL,
            context_index INTEGER NOT NULL
        )
        ''',
        '''
        CREATE TABLE IF NOT EXISTS streamings (
            user_id INTEGER NOT NULL REFERENCES users(user_id),
            track_id INTEGER NOT NULL REFERENCES tracks(track_id),
            timestamp TEXT NOT NULL,
            context_id INTEGER NOT NULL REFERENCES context(context_id)
        )
        '''
    ]

    for table in tables:
        postgres_cur.execute(table)
    postgres_conn.commit()

    # Connect to SQLite
    sqlite_conn = sqlite3.connect(sqlite_db_name)
    sqlite_cur = sqlite_conn.cursor()

    # Assuming the data insertion to SQLite is already done (from the provided function),
    # Let's extract data from SQLite and insert it into PostgreSQL

    for table_name in ["users", "albums", "artists", "tracks", "context", "streamings"]:
        sqlite_cur.execute(f"SELECT * FROM {table_name}")
        rows = sqlite_cur.fetchall()
        for row in rows:
            placeholders = ", ".join(["%s"] * len(row))
            postgres_cur.execute(f"INSERT INTO {table_name} VALUES ({placeholders})", row)
        postgres_conn.commit()

    sqlite_conn.close()
    postgres_conn.close()

# Just a placeholder for the JSON structure based on the function you provided
friends_activity_json_example = {
    'friends': [
        {
            'user': {'uri': 'spotify:user:123', 'name': 'John', 'imageUrl': 'https://example.com/image.jpg'},
            'track': {
                'uri': 'spotify:track:456',
                'name': 'Song Name',
                'imageUrl': 'https://example.com/track_image.jpg',
                'album': {'uri': 'spotify:album:789', 'name': 'Album Name'},
                'artist': {'uri': 'spotify:artist:101112', 'name': 'Artist Name'},
                'context': {'uri': 'spotify:context:131415', 'name': 'Context Name', 'index': 1}
            },
            'timestamp': '2022-01-01 10:00:00'
        }
    ]
}

# Uncomment the following line to run the function. Ensure the PostgreQSQL DB details are correct.

store_user_data_to_postgresql(main.friends_activity_json_example)


AttributeError: module 'main' has no attribute 'friends_activity_json_example'

In [3]:
from spotify_manager import *
redirect_uri = 'http://127.0.0.1:9090'
username = 'rt47etgc6xpwhhhb8575rth83'
client_ID = clint_id_secret.client_ID
client_SECRET = clint_id_secret.client_SECRET

# data_folder = r"C:\Users\saket\Documents\1.MY_DATA\spotify\spotify api data"
recently_played_file_name = 'recently_played_tracks.csv'

client_credentials_manager = spotipy.oauth2.SpotifyOAuth(
        # scope=scope,
        username=username,
        client_id=client_ID,
        client_secret=client_SECRET,
        redirect_uri=redirect_uri,
        # open_browser=False
        )

sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)
    

# returns a spotipy object with the given scope
def get_spotify_token(scope):
    client_credentials_manager.get_access_token(as_dict=False)
    return spotipy.Spotify(client_credentials_manager=client_credentials_manager, requests_timeout=10, retries=10)


""" ATTENTION you can find a better way, so that you don't have to get the every token everytime"""
# token to modify user's playlists
# playlist_modify_public = get_spotify_token("playlist-modify-public")

# user-read-recently-played
# user_read_recently_played = get_spotify_token("user-read-recently-played")
user_read_recently_played = spotipy.util.prompt_for_user_token(username, scope="user-read-recently-played")

# user-top-read
# user_top_read = get_spotify_token("user-top-read")

# playlist-modify-private
# playlist_modify_private = get_spotify_token("playlist-modify-private")

# user-library-read
# user_library_read = get_spotify_token("user-library-read")

# playlist-read-private
# playlist_read_private = get_spotify_token("playlist-read-private")

# get all the tracks from a private playlist
def get_playlist_tracks(playlist_id):
    """Get all the playlists that are new from the user's playlists info.

    Parameters
    ----------
    None

    Returns
    -------
    new_playlists : pd.DataFrame
        A data frame with the new playlists info and date.
    """

    results = playlist_read_private.user_playlist_tracks(username, playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    return tracks

""" ATTENTION needed here Need to be updated """
# retry 3 times to get the playlist tracks
# def get_playlist_tracks_retry(playlist_id):
#     """retry 3 times to get the playlist tracks

#     Parameters
#     ----------
#     playlist_id : str

#     Returns
#     -------
#     results : dict
#     """
#     for i in range(9999**9999):
#         try:
#             results = get_playlist_tracks(playlist_id)
#             return results
#         except Exception as e:
#             print(f"Error fetching playlist tracks: {e}")
#             print(f"Retrying in 10 seconds ({i+1}/20 attempts)...")
#             if i < 3:
#                 time.sleep(10)
#                 print('sleeping for 10 seconds')
#             elif i < 5:
#                 time.sleep(30)
#                 print('sleeping for 30 seconds')
#             else:
#                 time.sleep(60)
#                 print('sleeping for 60 seconds')
#     return None


def get_recently_played_tracks(limit=50):
    """ Get the recently played tracks from the user's account.

    parameters
    ----------
    limit : int (default=50)
        The number of tracks to return. Default: 50. Minimum: 1. Maximum: 50.
    
    Returns
    -------
    recently_played : dict
    """
    recently_played = user_read_recently_played.current_user_recently_played(limit=limit, )
    return recently_played

# def get_recently_played_tracks(limit=50):
#     """ Get the recently played tracks from the user's account.

#     parameters
#     ----------
#     limit : int (default=50)
#         The number of tracks to return. Default: 50. Minimum: 1. Maximum: 50.
    
#     Returns
#     -------
#     recently_played : dict
#         A dictionary containing the recently played tracks and their listening time.
#         The keys are 'tracks' and 'listening_time'.
#     """
#     recently_played = user_read_recently_played.current_user_recently_played(limit=limit, )
#     listening_time = sum([track['duration_ms'] for track in recently_played['items']])
#     return {'tracks': recently_played, 'listening_time': listening_time}

def add_to_playlist(playlist_id, list_of_tracks):
    """ Add the tracks to the playlist.

    Parameters
    ----------
    playlist_id : str
        The playlist id.

    list_of_tracks : list
        The list of tracks to add to the playlist.
    """
    splity_split = split_with_no(list_of_tracks, 100)
    for hundred_lis in splity_split:
        playlist_modify_private.user_playlist_add_tracks(user=username, playlist_id=playlist_id, tracks=hundred_lis)


def split_with_no(list, No):
    """splits the list into sublists of size 100

    Parameters
    ----------
    list : list
        The list to split.
    No : int
        The size of the sublists.

    Returns
    -------
    lists : list
    """
    if type(list) == str:
        return [list]
    lists = []
    no = 0
    listy = []
    for i in list:
        if no < 100:
            no += 1
            listy.append(i)
        else:
            lists.append(listy) 
            listy = []
            no = 0
    lists.append(listy)
    print('len - ',len(lists))
    return lists

# returns the top 50 tracks played by the user
def get_top_tracks(limit=50):
    """ Get the top tracks from the user's account.

    Parameters
    ----------
    limit : int (default=50)
        The number of tracks to return. Default: 50. Minimum: 1. Maximum: 50.

    Returns
    -------
    top_tracks : dict
    """
    top_tracks = token.current_user_top_tracks(limit=limit)
    return top_tracks

""" ATTENTION needed here Need to be updated """
# returns a datetime object from the given timestamp
def timestamp_to_time(timestamp):
    """returns a datetime object from the given timestamp

    Parameters
    ----------
    timestamp : str

    Returns
    -------
    dt_object : datetime.datetime
    """
    try:
        # Try to parse the timestamp string as ISO format with microsecond component
        dt_object = datetime.datetime.fromisoformat(timestamp[:-1])
    except ValueError:
        try:
            # Try to parse the timestamp string as ISO format without microsecond component
            dt_object = datetime.datetime.strptime(timestamp, '%Y-%m-%dT%H:%M:%SZ')
        except ValueError:
            # Try to parse the timestamp string as a Unix timestamp in seconds
            dt_object = datetime.datetime.fromtimestamp(int(timestamp))

    return dt_object

# returns a dictionary with the track info for the given track
def get_track_info_from_json(track_dict):
    """returns a dictionary with the track info for the given track

    Parameters
    ----------
    track_dict : dict

    Returns
    -------
    track_info : dict
    """
    # iterate through all the keys in the track_json try to get the value of the key, else set the value to None
    track_info = {}
    for key in track_dict.keys():
        try:
            track_info[key] = track_json[key]
        except:
            track_info[key] = None
    return track_info

def parse_my_streaming_history():

  # Call the get_recently_played_tracks function and store the result in a variable.
  data = get_recently_played_tracks()

  # Create an empty list to store the parsed data.
  parsed_data = []

  # Loop through the items in the data list.
  for item in data["items"]:

    # Get the track object.
    track = item["track"]

    # Get the artist/artists data.
    artists = track["artists"]
    artist_names = []
    artist_ids = []
    for artist in artists:
      artist_names.append(artist["name"])
      artist_ids.append(artist["id"])

    # Get the album data.
    album = track["album"]
    album_name = album["name"]
    album_id = album["id"]
    album_type = album["album_type"]
    release_date = album["release_date"]
    total_tracks = album["total_tracks"]

    # Get the image with 300 height from the images list.
    images = album["images"]
    image_300 = None
    for image in images:
      if image["height"] == 300:
        image_300 = image["url"]
        break

    # Get the track data.
    track_name = track["name"]
    track_number = track["track_number"]
    duration_ms = track["duration_ms"]
    disc_number = track["disc_number"]
    popularity = track["popularity"]
    preview_url = track["preview_url"]

    # Get the played_at data.
    played_at = item["played_at"]

    # Get the track id from the uri by removing the spotify:track: prefix.
    track_id = track["uri"].replace("spotify:track:", "")

    # Get the context data.
    
    try:
      context = item["context"]
      context_type = context["type"]
      context_uri = context["uri"]
    except:
      context_uri = None
      context_type = None

    # Create a dictionary to store the parsed data for each item.
    item_data = {
      # Artist details
      "artist_names": artist_names,
      "artist_uris": artist_ids,
      # Album details
      "album_name": album_name,
      "album_uri": album_id,
      "album_type": album_type,
      "release_date": release_date,
      "total_tracks": total_tracks,
      "image_url": image_300,
      # Track details
      "track_name": track_name,
      "track_number": track_number,
      "duration_ms": duration_ms,
      "disc_number": disc_number,
      "popularity": popularity,
      "preview_url": preview_url,
      "played_at": played_at,
      "track_uri": track_id,
      # Context details
      "context_type": context_type,
      "context_uri": context_uri
    }

    # Append the item data to the parsed data list.
    parsed_data.append(item_data)

  # Return the parsed data list.
  return parsed_data


# modify this function so that it could store the data to a sqllite database instead of csv
def get_top_tracks(limit=200):
    """returns the top tracks

    Parameters
    ----------
    limit : int, optional
        The number of entities to return. Default is 200. Minimum is 1. Maximum is 200(I guess # need to make sure)
    
    Returns
    -------
    top_tracks : dict
    """
    top_tracks = user_top_read.current_user_top_tracks(limit=limit, time_range='short_term')
    return top_tracks

# returns the top tracks info in a list
def get_top_tracks_info():
    top_tracks = get_top_tracks()
    top_tracks_info = []
    for i in range(len(top_tracks['items'])):
        track = top_tracks['items'][i]
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id']
        }
        top_tracks_info.append(track_info)
    return top_tracks_info

def get_top_artists_info(limit=200):
    """returns the top artists info in a list

    Parameters
    ----------
    limit : int, optional
        The number of entities to return.

    Returns
    -------
    top_artists_info : list
    """
    top_artists = user_top_read.current_user_top_artists(limit=limit)
    top_artists_info = []
    for i in range(len(top_artists['items'])):
        artist = top_artists['items'][i]
        artist_info = {
            'artist_name': artist['name'],
            'artist_id': artist['id'],
            'artist_popularity': artist['popularity'],
            'artist_followers': artist['followers']['total'],
            'artist_genres': artist['genres']
        }
        top_artists_info.append(artist_info)
    return top_artists_info

# returns the user's top genres in a list
def get_top_genres_info(limit=200):
    """returns the user's top genres in a list

    Parameters
    ----------
    limit : int, optional
        The number of entities to return.

    Returns
    -------
    top_genres_info : list
    """
    top_artists = user_top_read.current_user_top_artists(limit=limit)
    top_genres_info = []
    for i in range(len(top_artists['items'])):
        artist = top_artists['items'][i]
        for j in range(len(artist['genres'])):
            genre = artist['genres'][j]
            genre_info = {
                'genre': genre
            }
            top_genres_info.append(genre_info)
    return top_genres_info

# returns the user's library tracks in a list
def get_library_info(limit=200):
    """returns the user's library tracks in a list

    Parameters
    ----------
    limit : int, optional
        The number of entities to return.

    Returns
    -------
    library_info : list
    """
    # get the token for the user-library-read scope
    # get the user's library data
    library = user_library_read.current_user_saved_tracks(limit=limit)

    # convert the library data to a dataframe
    library_df = pd.DataFrame(library)
    return library_df

# returns a list of dictionaries containing the user's playlists
def get_user_playlists():
    """ returns a list of dictionaries containing the user's playlists

    Returns
    -------
    user_playlists : list
        list of dictionaries containing the user's playlists
    """
    user_playlists = playlist_read_private.current_user_playlists(limit=50)
    return user_playlists

# # returns a list of dictionaries containing the user's playlists
# def store_user_playlists_info():
#     user_playlists_info = get_user_playlists_info()
#     user_playlists_info_df = pd.DataFrame(user_playlists_info)
#     user_playlists_info_df.to_csv('user_playlists_info.csv', index=False)

# returns the tracks in a playlist in a dictionary
def get_user_playlist_tracks(playlist_id):
    """ returns the tracks in a playlist in a dictionary

    Parameters
    ----------
    playlist_id : str
        The Spotify ID for the playlist.

    Returns
    -------
    user_playlist_tracks : dict
        The tracks in a playlist in a dictionary
    """
    user_playlist_tracks = playlist_read_private.user_playlist_tracks(playlist_id)
    return user_playlist_tracks

""" ATTENTION: This function is not working properly."""
def sort_playlist_by_popularity():
    """ Sorts the playlist by popularity

    Returns
    -------
    sorted_playlist : list
        The tracks in a playlist sorted by popularity
    """
    user_playlist_tracks = get_user_playlist_tracks(playlist_id)
    sorted_playlist = []
    for i in range(len(user_playlist_tracks['items'])):
        track = user_playlist_tracks['items'][i]['track']
        popularity = track['popularity']
        sorted_playlist.append(popularity)
    sorted_playlist.sort(reverse=True)
    return sorted_playlist


# returns the tracks in a playlist in a list of dictionaries
def get_user_playlist_tracks_info(playlist_id):
    """ returns the tracks in a playlist in a list of dictionaries

    Parameters
    ----------
    playlist_id : str
        The Spotify ID for the playlist.

    Returns
    -------
    user_playlist_tracks_info : list
        The tracks in a playlist in a list of dictionaries
    """
    user_playlist_tracks = get_user_playlist_tracks(playlist_id)
    user_playlist_tracks_info = []
    for i in range(len(user_playlist_tracks['items'])):
        track = user_playlist_tracks['items'][i]['track']
        return track
        track_info = {
            'track_name': track['name'],
            'track_id': track['id'],
            'track_duration': track['duration_ms'],
            'track_popularity': track['popularity'],
            'track_explicit': track['explicit'],
            'album_name': track['album']['name'],
            'album_id': track['album']['id'],
            'album_release_date': track['album']['release_date'],
            'album_release_date_precision': track['album']['release_date_precision'],
            'album_total_tracks': track['album']['total_tracks'],
            'album_type': track['album']['album_type'],
            'artist_name': track['artists'][0]['name'],
            'artist_id': track['artists'][0]['id']
        }
        user_playlist_tracks_info.append(track_info)
    return user_playlist_tracks_info

# returns a json of recommended tracks according to the seed artists, genres and tracks
def get_recommendations(seed_artists, seed_genres, seed_tracks):
    """ returns a json of recommended tracks according to the seed artists, genres and tracks

    Parameters
    ----------
    seed_artists : list
        A list of Spotify ID/IDs for seed artists.
    seed_genres : list
        A list of any genres in the set of available genre seeds.
    seed_tracks : list
        A list of Spotify ID/IDs for seed tracks.

    Returns
    -------
    recommendations : dict
        A json of recommended tracks according to the seed artists, genres and tracks
    """
    recommendations = user_top_read.recommendations(seed_artists=seed_artists, seed_genres=seed_genres, seed_tracks=seed_tracks)
    return recommendations

def get_recommendations_info(seed_artists, seed_genres, seed_tracks):
    """ returns a list of dictionaries containing the important info from recommendations json

    Parameters
    ----------
    seed_artists : list
        A list of Spotify ID/IDs for seed artists.
    seed_genres : list
        A list of any genres in the set of available genre seeds.
    seed_tracks : list
        A list of Spotify ID/IDs for seed tracks.

    Returns
    -------
    recommendations_info : list
        A list of dictionaries containing the important info from recommendations json
    """
    # get the json of the recommended tracks
    recommendations = get_recommendations(seed_artists, seed_genres, seed_tracks)
    recommendations_info = []
    # get the info of each track
    for i in range(len(recommendations['tracks'])):
        track = recommendations['tracks'][i]
        track_info = get_track_info_from_json(track)
        recommendations_info.append(track_info)
    return recommendations_info

# returns the user's saved albums in a list
def get_saved_albums():
    """
    Returns
    -------
    saved_albums : list
        The user's saved albums in a list
    """
    saved_albums = user_library_read.current_user_saved_albums()
    return saved_albums

# returns the user's saved shows in a list
def get_saved_shows():
    """
    Returns
    -------
    saved_shows : list
        The user's saved shows in a list
    """
    saved_shows = user_library_read.current_user_saved_shows()
    return saved_shows

# returns the user's saved episodes in a list
def get_saved_episodes():
    """
    Returns
    -------
    saved_episodes : list
        The user's saved episodes in a list
    """
    saved_episodes = user_library_read.current_user_saved_episodes()
    return saved_episodes

def get_library_info(limit):
    """
    Parameters
    ----------
    limit : int
        The number of tracks to return. Default: 20. Minimum: 1. Maximum: 50.
    
    Returns
    -------
    library_df : pandas dataframe
        The user's library data in a pandas dataframe
    """
    # get the user's library data
    library = user_library_read.current_user_saved_tracks(limit=limit)
    # convert the library data to a dataframe
    library_df = pd.DataFrame(library)
    return library_df

# Define a function to get the value of a nested key from a dictionary
def get_nested_value(dictionary, keys, default=''):
    """ Returns the value of a nested key from a dictionary

    Parameters
    ----------
    dictionary : dict
        The dictionary to get the value from
    keys : list
        A list of keys to get the value from
    default : str
        The default value to return if the key is not found

    Returns
    -------
    dictionary : dict
        The dictionary to get the value from
    keys : list
        A list of keys to get the value from
    default : str
        The default value to return if the key is not found
    """
    # Loop through the keys
    for key in keys:
        # Try to get the value of the current key
        try :
            dictionary = dictionary[key]
        # If the key is not found, return the default value
        except :
            return default
    # Return the final value
    return dictionary

# returns a list of dictionaries containing the user's playlists information
def get_user_playlists_info():
    """ Returns a list of dictionaries containing the user's playlists information
    Parameters:
    ----------
    None

    Returns:
    -------
    user_playlists_info: list of dictionaries
    """
    user_playlists = get_user_playlists()
    user_playlists_info = []
    for i in range(len(user_playlists['items'])):
        playlist = user_playlists['items'][i]
        playlist_info = {
            'playlist_name': playlist['name'],
            'playlist_id': playlist['id'],
            'playlist_description': playlist['description'],
            'playlist_tracks': playlist['tracks']['total'],
            'playlist_owner': playlist['owner']['display_name'],
            'playlist_owner_id': playlist['owner']['id'],
            'playlist_public': playlist['public'],
            'playlist_collaborative': playlist['collaborative'],
        }
        if playlist.get('images'):
            playlist_info['playlist_images'] = playlist['images'][0].get('url', '')
        else:
            playlist_info['playlist_images'] = ''
        # playlist_info['playlist_images'] = playlist.get('images', [{}])[0].get('url', '')

        user_playlists_info.append(playlist_info)
    return user_playlists_info


# """ ATTENTION NEEDED HERE"""
# # store the user's playlists information in a csv file
# def store_user_playlists_info():
#     """change to to store into to a sql database
#     """
#     user_playlists_info = get_user_playlists_info()
#     df = pd.DataFrame(user_playlists_info)
#     # add a column with the current date and time
#     df['date'] = datetime.datetime.now()
#     df.to_csv('user_playlists_info.csv', index=False)

# get the commands stored in the playlist name or description
def gather_functions(string):
    """ uses regex to get the commands stored in the playlist name or description

    Parameters
    ----------
    string : str
        The string to get the functions from

    Returns
    -------
    functions : list
        A list of functions that contains a list of words(commands) to be performed on the playlist
    """
    # lowercases the string
    test = string.lower()
    # write a regex to get the artist which should return swans
    artist = re.search(r'artist-(\w+(?:-\w+)*)', string)
    # write a regex to get the gather which should return gather-all
    gather = re.search(r'gather-(\w+(?:-\w+)*)', string)

    # write a regex to get the genre which should return hip-hop
    genre = re.search(r'genre-(\w+(?:-\w+)*)', string)
    # # write a regex to get the words that has " -" after it
    # words_functions = re.findall(r'(\w+(?:-\w+)*) -', test)
    # dic = {"genre": genre.group(1), "gather": gather.group(1), "artist": artist.group(1)}
    dic = {"genre": None, "gather": None, "artist": None}
    if genre:
        dic["genre"] = genre.group(1).replace("_", " ")
    if gather:
        dic["gather"] = gather.group(1).replace("_", " ")
    if artist:
        dic["artist"] = artist.group(1).replace("_", " ")
    return dic

# # get all the playlists that are new
# def get_new_playlists():

#     user_playlist_info = get_user_playlists_info()
#     new_df = pd.DataFrame(user_playlist_info)
#     new_df['date'] = datetime.datetime.now()
#     # read the csv file
#     old_df = pd.read_csv('user_playlists_info.csv')

#     """Iterate though the new_df and search for matching value from the old dataframe save the index where the first value matches. the value should continue to match."""
#     # iterate through the new_df
#     user_playlist_info = get_user_playlists_info()
#     new_df = pd.DataFrame(user_playlist_info)
#     new_df['date'] = datetime.datetime.now()
#     # read the csv file
#     old_df = pd.read_csv('user_playlists_info.csv')
#     the_index = 0
#     previous_index = -1
#     for i in range(len(new_df)):
#         song_id = new_df['playlist_id'][i]
#         try:
#             # get the index of the matching value
#             index = old_df[old_df['playlist_id'] == song_id].index[0]
#             # if the index is not the previous index + 1 and it is not the first index
#             if i != previous_index+1 and i != 0:
#                 # save the index
#                 the_index = i
#             previous_index = i
#         except IndexError:
#             pass
#     changed = new_df[:the_index]

#     # replace the rows before the index with the new rows, changed
#     old_df[:the_index] = changed
#     # save the new dataframe to the csv file
#     # old_df.to_csv('user_playlists_info.csv', index=False)
#     # return all the playlits that are new
#     # store_user_playlists_info() # pause while updating the function
#     return changed

def get_new_playlists():
    """Get all the playlists that are new from the user's playlists info.

    Parameters
    ----------
    None

    Returns
    -------
    new_playlists : pd.DataFrame
        A data frame with the new playlists info and date.
    """
    # Get the user's playlists info as a data frame
    user_playlist_info = get_user_playlists_info()
    new_df = pd.DataFrame(user_playlist_info)
    # Add the current date column
    new_df['date'] = datetime.datetime.now()
    # Read the csv file as a data frame
    old_df = pd.read_csv('user_playlists_info.csv')
    # Merge the two data frames on playlist id
    merged_df = pd.merge(new_df, old_df, on='playlist_id', how='outer', indicator=True)
    # Filter the data frame to keep only the rows that are in the left data frame (new_df)
    new_playlists = merged_df[merged_df['_merge'] == 'left_only']
    # Drop the _merge column
    new_playlists.drop('_merge', axis=1, inplace=True)
    # Return the new playlists data frame
    return new_playlists


# # get all the playlists with functions hidden in the name or description
# def get_playlists_with_functions():
#     new_playlists = get_new_playlists()
#     # create a dictonary to store the playlists id with functions
#     playlists_with_functions = {}
#     # iterate through the new playlists
#     for i in range(len(new_playlists)):
#         # use gather_functions to check if the playlist name and description has functions
#         name = gather_functions(new_playlists['playlist_name'][i].lower())
#         description = gather_functions(new_playlists['playlist_description'][i].lower())
#         funciton_data = {}
#         # print(name)
#         # print(description)
#         # iterate through the dictonary and check if the value is not None
#         for key, value in name.items():
#             if value:
#                 # if both the name and description have different values for the same key set a list with both values
#                 if description[key] and description[key] != value:
#                     funciton_data[key] = [value, description[key]]
#                 else:
#                     funciton_data[key] = [value]
#             # add playlist id to the dictonary
#             # funciton_data['playlist_id'] = new_playlists['playlist_id'][i]
#         # save funciton_data to the playlists_with_functions dictonary with key as the playlist id
#         playlists_with_functions[new_playlists['playlist_id'][i]] = funciton_data
#     return playlists_with_functions

def get_playlists_with_functions():
    """Get new playlists with functions from their names and descriptions.

    Parameters
    ----------
    None

    Returns
    -------
    playlists_with_functions : dict
        A dictionary with playlist ids as keys and function data as values.
    """
    new_playlists = get_new_playlists()
    # Create a dictionary to store the playlists id with functions
    playlists_with_functions = {}
    # Iterate through the new playlists
    for i, playlist in enumerate(new_playlists):
        # Use gather_functions to check if the playlist name and description has functions
        name = gather_functions(playlist['playlist_name'].lower())
        description = gather_functions(playlist['playlist_description'].lower())
        function_data = {}
        # Iterate through the dictionary and check if the value is not None
        for key, value in name.items():
            # If both the name and description have different values for the same key set a list with both values
            if description.get(key) and description.get(key) != value:
                function_data[key] = [value, description.get(key)]
            else:
                function_data[key] = [value]
        # Save function_data to the playlists_with_functions dictionary with key as the playlist id
        playlists_with_functions[playlist['playlist_id']] = function_data
    return playlists_with_functions

# # search for aritst and return the artist id
# def search_for_artist(artist):
#     # search for the artist
#     results = sp.search(q='artist:' + artist, type='artist')
#     artist_id = results['artists']['items'][0]['id']
#     return artist_id

def search_for_artist(artist):
    """Search for an artist on Spotify and return their id.

    Parameters
    ----------
    artist : str
        The name of the artist.

    Returns
    -------
    artist_id : str or None
        The id of the artist, or None if no match is found or multiple matches are found.
    """
    # Search for the artist
    results = sp.search(q=f"artist:{artist}", type='artist')
    # Check if the search returned exactly one result
    if len(results['artists']['items']) == 1:
        # Return the id of the artist
        artist_id = results['artists']['items'][0]['id']
        return artist_id
    else:
        # Return None and print a message
        print(f"No match or multiple matches found for {artist}.")
        return None

# get the all the tracks from the artist
# def get_artist_discography(name):
#     # Get the artist's albums
#     results = sp.artist_albums(search_for_artist(name), album_type='album')
#     # Extract relevant data from the API response
#     all_tarck_ids = []
#     for album in results['items']:

#         # Get the tracks of the album
#         album_tracks = sp.album_tracks(album['id'])

#         # Extract relevant data from the API response
#         for idx, track in enumerate(album_tracks['items']):
#             print(f"Track {idx+1}: {track['name']} by {track['artists'][0]['name']}")
#             print(f"Track ID: {track['id']}")
#             all_tarck_ids.append(track['id'])
#     return all_tarck_ids

def get_artist_discography(name):
    """Get the artist's discography from Spotify and print the track names and ids.

    Parameters
    ----------
    name : str
        The name of the artist.

    Returns
    -------
    all_track_ids : list
        A list of track ids from the artist's albums.
    """
    # Get the artist's albums
    results = sp.artist_albums(search_for_artist(name), album_type='album')
    # Extract relevant data from the API response
    all_track_ids = []
    for album in results['items']:

        # Get the tracks of the album
        album_tracks = sp.album_tracks(album['id'])

        # Extract relevant data from the API response
        all_track_ids += [track['id'] for track in album_tracks['items']]
        for idx, track in enumerate(album_tracks['items']):
            print(f"Track {idx+1}: {track['name']} by {track.__name__}")
            print(f"Track ID: {track['id']}")
    return all_track_ids


r = get_recently_played()

print(r)


                You need to set your Spotify API credentials.
                You can do this by setting environment variables like so:

                export SPOTIPY_CLIENT_ID='your-spotify-client-id'
                export SPOTIPY_CLIENT_SECRET='your-spotify-client-secret'
                export SPOTIPY_REDIRECT_URI='your-app-redirect-url'

                Get your credentials at
                    https://developer.spotify.com/my-applications
            


SpotifyException: http status: 550, code:-1 - no credentials set, reason: None

In [2]:
from main import *


                You need to set your Spotify API credentials.
                You can do this by setting environment variables like so:

                export SPOTIPY_CLIENT_ID='your-spotify-client-id'
                export SPOTIPY_CLIENT_SECRET='your-spotify-client-secret'
                export SPOTIPY_REDIRECT_URI='your-app-redirect-url'

                Get your credentials at
                    https://developer.spotify.com/my-applications
            


SpotifyException: http status: 550, code:-1 - no credentials set, reason: None

In [7]:

# """ ATTENTION you can find a better way, so that you don't have to get the every token everytime"""
# # token to modify user's playlists
# playlist_modify_public = get_spotify_token("playlist-modify-public")

# # user-read-recently-played
user_read_recently_played = get_spotify_token("user-read-recently-played")

# # user-top-read
# user_top_read = get_spotify_token("user-top-read")

# # playlist-modify-private
# playlist_modify_private = get_spotify_token("playlist-modify-private")

# # user-library-read
# user_library_read = get_spotify_token("user-library-read")

# # playlist-read-private
# playlist_read_private = get_spotify_token("playlist-read-private")


get_recently_played_tracks()

{'items': [{'track': {'album': {'album_type': 'album',
     'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/3u1ulLq00Y3bfmq9FfjsPu'},
       'href': 'https://api.spotify.com/v1/artists/3u1ulLq00Y3bfmq9FfjsPu',
       'id': '3u1ulLq00Y3bfmq9FfjsPu',
       'name': 'Ween',
       'type': 'artist',
       'uri': 'spotify:artist:3u1ulLq00Y3bfmq9FfjsPu'}],
     'available_markets': ['AD',
      'AE',
      'AG',
      'AL',
      'AM',
      'AO',
      'AR',
      'AT',
      'AU',
      'AZ',
      'BA',
      'BB',
      'BD',
      'BE',
      'BF',
      'BG',
      'BH',
      'BI',
      'BJ',
      'BN',
      'BO',
      'BR',
      'BS',
      'BT',
      'BW',
      'BY',
      'BZ',
      'CA',
      'CD',
      'CG',
      'CH',
      'CI',
      'CL',
      'CM',
      'CO',
      'CR',
      'CV',
      'CW',
      'CY',
      'CZ',
      'DE',
      'DJ',
      'DK',
      'DM',
      'DO',
      'DZ',
      'EC',
      'EE',
      'EG',
      'ES'